In [2]:
import torch
import torchvision
import lightning
from torchvision.models.detection import MaskRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
from torchvision.datasets import Cityscapes
from torch.utils.data import DataLoader

In [3]:
DATA = '/home/gtangg12/auto-augment/data'

In [4]:
class MaskRCNNModule(lightning.LightningModule):
    """
    """
    def __init__(self, num_classes):
        super().__init__()
        self.model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

        # Replace the classifier and mask predictor
        in_features      = self.model.roi_heads.box_predictor.cls_score  .in_features
        in_features_mask = self.model.roi_heads.mask_predictor.conv5_mask.in_channels
        self.model.roi_heads.box_predictor  = FastRCNNPredictor(in_features          , num_classes)
        self.model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask, 256, num_classes)

        self.transform = torchvision.transforms.Compose([
            torchvision.transforms.ToTensor(),
            torchvision.transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                             std =[0.229, 0.224, 0.225])
        ])

    def forward(self, images, targets=None):
        return self.model(images, targets)

    def training_step(self, batch, batch_idx):
        images, targets = batch
        loss_dict = self(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        return loss

    def validation_step(self, batch, batch_idx):
        images, targets = batch
        loss_dict = self(images, targets)
        loss = sum(loss for loss in loss_dict.values())
        self.log('val_loss', loss)
        return loss

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.parameters(), lr=0.0005)
        return optimizer

In [8]:
train_dataset, val_dataset = \
    Cityscapes(DATA, split='train', mode='fine', target_type='semantic'), \
    Cityscapes(DATA, split='val'  , mode='fine', target_type='semantic')

/home/gtangg12/auto-augment/data


RuntimeError: Dataset not found or incomplete. Please make sure all required folders for the specified "split" and "mode" are inside the "root" directory

In [6]:
model = MaskRCNNModule(num_classes=21)
trainer = lightning.Trainer(accelerator='gpu', max_epochs=30)
trainer.fit(model)

/home/gtangg12/mambaforge/envs/auto-augment/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/gtangg12/mambaforge/envs/auto-augment/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=MaskRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=MaskRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/gtangg12/mambaforge/envs/auto-augment/lib/python3.10/site-packages/lightning/pytorch/trainer/connectors/logger_connector/logger_conne

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

RuntimeError: Dataset not found or incomplete. Please make sure all required folders for the specified "split" and "mode" are inside the "root" directory